In [2]:
!pip install surprise -q -q -q
from surprise import KNNBasic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
import pandas as pd

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [29]:
train_data = pd.read_csv('/content/gdrive/MyDrive/Recsys_data/train.csv', error_bad_lines=False, usecols=['msno','song_id','target'])
# members_info=pd.read_csv('/content/gdrive/MyDrive/Recsys_data/members.csv')
# test_data=pd.read_csv('/content/gdrive/MyDrive/Recsys_data/test.csv')

<ipython-input-29-84065b51710d>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  train_data = pd.read_csv('/content/gdrive/MyDrive/Recsys_data/train.csv', error_bad_lines=False, usecols=['msno','song_id','target'])
<ipython-input-29-84065b51710d>:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('/content/gdrive/MyDrive/Recsys_data/train.csv', error_bad_lines=False, usecols=['msno','song_id','target'])


In [30]:
train_data.dropna(inplace = True)
train_data.reset_index(inplace = True)

In [46]:
train_data['msno'][8]

'uHqAtShXTRXju5GE8ri3ITsVFepPf8jUoCF7ffNOuqE='

In [51]:

from sklearn.preprocessing import MinMaxScaler

In [67]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

train_data = train_data[train_data['target'].astype(str).str.isdigit()]
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(train_data[['msno', 'song_id', 'target']], reader)



In [70]:

from surprise.model_selection import GridSearchCV


# Choosing the parameters for grid
sim_options = {
    'name': 'pearson_baseline',  # Choose the similarity measure, e.g., cosine, pearson
    'user_based': True  # Specify whether to use user-based or item-based collaborative filtering
}
#Knn basic object
knn_basic = KNNBasic(sim_options=sim_options,normalize=True, user_based=True,)



Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [74]:
from surprise import Dataset
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

trainset, testset = train_test_split(data, test_size=0.25, )
knn_basic.fit(trainset)

predictions = knn_basic.test(testset)

accuracy.rmse(predictions)

user_id = 'uHqAtShXTRXju5GE8ri3ITsVFepPf8jUoCF7ffNOuqE='

user_inner_id = trainset.to_inner_uid(user_id)

k_nearest_neighbors = knn_basic.get_neighbors(user_inner_id, k=10)

item_ratings = {}

for neighbor_inner_id in k_nearest_neighbors:
    neighbor_raw_id = trainset.to_raw_uid(neighbor_inner_id)

    neighbor_ratings = trainset.ur[neighbor_inner_id]

    for item_inner_id, rating in neighbor_ratings:
        item_raw_id = trainset.to_raw_iid(item_inner_id)

        if item_raw_id not in item_ratings:
            item_ratings[item_raw_id] = rating
        else:
            item_ratings[item_raw_id] = (item_ratings[item_raw_id] + rating) / 2

top_n_recommendations = sorted(item_ratings.items(), key=lambda x: x[1], reverse=True)[:10]

#top N recommendations
for song_id, predicted_rating in top_n_recommendations:
    print(f'Song ID: {song_id}, target: {predicted_rating}')



Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.4083
Song ID: /y60wRjvYtUHNYg1jjAU3XscE+BN9Z74p7pfYQ+xjfM=, target: 1.0
Song ID: SJOEwaBBHX3xNy0Cu6CPoqxMAs7cuJ26v8i2yHuvELk=, target: 1.0
Song ID: GQqErVwbH/YaifR6wl2S6prosRVHhwGO2/Aq5aIJLA8=, target: 1.0
Song ID: GjIfPZMlRbZcUhy+bXvh92yoO2imByIIKQrsBT/Hook=, target: 1.0
Song ID: cxlWjHAhKqO2bHxWa3uemzScKrOfXqcycSi4r87yT0Y=, target: 1.0
Song ID: 3Yz717mUtR+r4L0zzgDgWnYeFtY9g68ZjKlq1U72KNw=, target: 1.0
Song ID: 537ZQfKiswTVTs41yPIiIVz338Jw58+Ym7nhRkjivXc=, target: 1.0
Song ID: +QHBcF9ei9nhEJl9A+385mP0ztuBqVG6R4vijsNx7Uw=, target: 1.0
Song ID: zuRGGQHOiNCAEoHguelhLxy5EqPNpX4zywv6/uxDbcQ=, target: 1.0
Song ID: kmvJKGLVtgymT+JiPJkxffB1eIga2TYaziuZm1ozL+U=, target: 1.0
